In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [3]:
columns = ["id", "cycle", "op1", "op2", "op3", "sensor1", "sensor2", "sensor3", "sensor4", "sensor5", 
           "sensor6", "sensor7", "sensor8", "sensor9", "sensor10", "sensor11", "sensor12", "sensor13", 
           "sensor14", "sensor15", "sensor16", "sensor17", "sensor18", "sensor19", "sensor20", "sensor21", 
           "sensor22", "sensor23"]

In [6]:
train = pd.read_csv(r"C:\Users\Admin\Desktop\Chinmayee Verma\Turbofan Predictive Maintainance (NASA) (Dataset)\Dataset\train_FD001.txt", sep=" ", names=columns, engine='python')
test = pd.read_csv(r"C:\Users\Admin\Desktop\Chinmayee Verma\Turbofan Predictive Maintainance (NASA) (Dataset)\Dataset\test_FD001.txt", sep=" ", names=columns, engine='python')
test_result = pd.read_csv(r"C:\Users\Admin\Desktop\Chinmayee Verma\Turbofan Predictive Maintainance (NASA) (Dataset)\Dataset\RUL_FD001.txt", sep=" ", header=None, engine='python')

In [7]:
print(train.head())
print(train.info())
print(train.describe())

   id  cycle     op1     op2    op3  sensor1  sensor2  sensor3  sensor4  \
0   1      1 -0.0007 -0.0004  100.0   518.67   641.82  1589.70  1400.60   
1   1      2  0.0019 -0.0003  100.0   518.67   642.15  1591.82  1403.14   
2   1      3 -0.0043  0.0003  100.0   518.67   642.35  1587.99  1404.20   
3   1      4  0.0007  0.0000  100.0   518.67   642.35  1582.79  1401.87   
4   1      5 -0.0019 -0.0002  100.0   518.67   642.37  1582.85  1406.22   

   sensor5  ...  sensor14  sensor15  sensor16  sensor17  sensor18  sensor19  \
0    14.62  ...   8138.62    8.4195      0.03       392      2388     100.0   
1    14.62  ...   8131.49    8.4318      0.03       392      2388     100.0   
2    14.62  ...   8133.23    8.4178      0.03       390      2388     100.0   
3    14.62  ...   8133.83    8.3682      0.03       392      2388     100.0   
4    14.62  ...   8133.80    8.4294      0.03       393      2388     100.0   

   sensor20  sensor21  sensor22  sensor23  
0     39.06   23.4190       Na

In [8]:
operational_cols = ["op1", "op2", "op3"]
sensor_cols = [col for col in train.columns if 'sensor' in col]

print("Missing values in train:\n", train.isnull().sum())
imputer = SimpleImputer(strategy='median')
train[sensor_cols] = imputer.fit_transform(train[sensor_cols])

for col in sensor_cols:
    sns.boxplot(x=train[col])
    plt.title(f"Box plot for {col}")
    plt.show()

Missing values in train:
 id              0
cycle           0
op1             0
op2             0
op3             0
sensor1         0
sensor2         0
sensor3         0
sensor4         0
sensor5         0
sensor6         0
sensor7         0
sensor8         0
sensor9         0
sensor10        0
sensor11        0
sensor12        0
sensor13        0
sensor14        0
sensor15        0
sensor16        0
sensor17        0
sensor18        0
sensor19        0
sensor20        0
sensor21        0
sensor22    20631
sensor23    20631
dtype: int64


c:\Users\Admin\anaconda3\envs\ChinmayeeENV\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: ['sensor22' 'sensor23']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Columns must be same length as key

In [9]:
# Apply IQR method to cap outliers
for col in sensor_cols:
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    train[col] = train[col].clip(lower_bound, upper_bound)


In [10]:
# 4. Feature Engineering
# Calculate Remaining Useful Life (RUL)
rul_train = train.groupby('id')['cycle'].max().reset_index()
rul_train.columns = ['id', 'MaxCycle']
train = train.merge(rul_train, on='id')
train['RUL'] = train['MaxCycle'] - train['cycle']
train.drop(columns=['MaxCycle'], inplace=True)

# Check correlations with RUL
correlations = train.corr()['RUL'].drop('RUL')
important_features = correlations[correlations.abs() > 0.1]
print("Important Features for Predicting RUL:\n", important_features)


Important Features for Predicting RUL:
 cycle      -0.736241
sensor2    -0.606951
sensor3    -0.585138
sensor4    -0.679527
sensor7     0.657792
sensor8    -0.565185
sensor9    -0.365960
sensor11   -0.697033
sensor12    0.672671
sensor13   -0.563890
sensor14   -0.252565
sensor15   -0.643236
sensor17   -0.606843
sensor20    0.629971
sensor21    0.636204
Name: RUL, dtype: float64


In [11]:
# 5. Normalization and Feature Scaling
# Choose scaling based on model needs; here we use Min-Max Scaling
scaler = MinMaxScaler()
train[sensor_cols] = scaler.fit_transform(train[sensor_cols])


c:\Users\Admin\anaconda3\envs\ChinmayeeENV\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\Admin\anaconda3\envs\ChinmayeeENV\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [12]:
# 6. Encoding Categorical Variables
# In this case, operational settings could be categorical, so let's check if they need encoding
# Assuming op1, op2, and op3 might be categorical; encoding them
encoder = OneHotEncoder(sparse=False)
op_encoded = encoder.fit_transform(train[operational_cols])
op_encoded_df = pd.DataFrame(op_encoded, columns=encoder.get_feature_names_out(operational_cols))
train = pd.concat([train.reset_index(drop=True), op_encoded_df.reset_index(drop=True)], axis=1)
train.drop(columns=operational_cols, inplace=True)


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [13]:
# 7. Target Variable Processing
# If target RUL needs further transformation, adjust as needed here. 
# (In this case, we already have RUL calculated and scaled, so no further steps are necessary.)

In [ ]:
# 8. Final Data Review and Saving
# Check for any leakage, e.g., ensuring RUL calculation is accurate and independent of future cycle
train = train.sort_values(['id', 'cycle'])

# Save preprocessed data for future use
train.to_csv("processed_train.csv", index=False)
test.to_csv("processed_test.csv", index=False)
test_result.to_csv("processed_test_result.csv", index=False)
